<a href="https://colab.research.google.com/github/Alilson2/Projeto_IA/blob/main/ProjetoIA_ver_David.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr # Ler arquivos netcdf
from google.colab import drive
import seaborn as sns

!pip install cartopy
import cartopy

import cartopy.crs as ccrs # Escolha do sistema de coordenadas
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.patches as mpatches # Desenhar geometria em um mapa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 29.1 MB/s eta 0:00:00


In [2]:
import os
import xarray as xr

# --- Clonar repositório se não existir ---
if not os.path.exists("Projeto_IA"):
    !git clone https://github.com/Alilson2/Projeto_IA.git
else:
    print("📁 Repositório 'Projeto_IA' já existe — pulando o clone.")

# --- Verificar se a pasta foi criada ---
if not os.path.exists("Projeto_IA"):
    raise FileNotFoundError("❌ A pasta 'Projeto_IA' não foi encontrada. O clone pode ter falhado.")
else:
    print("\n✅ Repositório clonado com sucesso!\n")
    print("Arquivos dentro da pasta Projeto_IA:\n", os.listdir("Projeto_IA"))

# --- Localizar arquivos .nc ---
arquivos_nc = [f for f in os.listdir("Projeto_IA") if f.endswith(".nc")]
if not arquivos_nc:
    raise FileNotFoundError("❌ Nenhum arquivo .nc encontrado na pasta Projeto_IA!")
else:
    print("\n📂 Arquivo(s) NetCDF encontrado(s):")
    for f in arquivos_nc:
        print(" -", f)

# --- Montar lista de caminhos ---
ARQUIVO = [os.path.join("Projeto_IA", f) for f in arquivos_nc]

# --- Função para corrigir longitude ---
def corrigir_longitude(ds):
    for coord in ["longitude", "lon"]:
        if coord in ds.coords:
            ds = ds.assign_coords({coord: ((ds[coord] + 180) % 360) - 180})
            ds = ds.sortby(coord)
    return ds

# --- Abrir arquivos com segurança (nova sintaxe) ---
try:
    dados = xr.open_mfdataset(
        ARQUIVO,
        combine='by_coords',
        parallel=True,           # usa múltiplos núcleos
        preprocess=corrigir_longitude,
        combine_attrs='override' # 🟢 substitui o antigo compat='override'
    )
except ValueError as e:
    print("\n⚠️ Erro na combinação — tentando modo 'nested' (concat por tempo)...")
    dados = xr.open_mfdataset(
        ARQUIVO,
        combine='nested',
        concat_dim='valid_time',  # ajuste se sua dimensão temporal tiver outro nome
        parallel=True,
        preprocess=corrigir_longitude,
        combine_attrs='override'
    )

print("\n✅ Dataset carregado com sucesso!\n")
dados

Cloning into 'Projeto_IA'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 190 (delta 36), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (190/190), 142.51 MiB | 18.00 MiB/s, done.
Resolving deltas: 100% (36/36), done.

✅ Repositório clonado com sucesso!

Arquivos dentro da pasta Projeto_IA:
 ['(FEVEREIRO 2023 - 16a28) 7dc44bd550a6fed4a3ceee541998bb8c.nc', 'MAI 2022 - 16a31.nc', '(NOV 2020-1 a 24).nc', '(DEZ 2020-25 a 31).nc', '(MARÇO 2024 - 1a15) 27b35cf2be7d06ee99adbf9b6c8a7e3d.nc', '(MARÇO 2023 - 1a15) ef5ac9bd28416b585bee81ec1bb1651a.nc', 'JAN 2022 - 16a31.nc', 'SET 2022 - 16a30.nc', 'FEV 2022 - 16a28.nc', '(MAIO 2021-1 a 15).nc', '(Junho 2021-1 a 15).nc', '(ABRIL 2023 - 16a30) 13b26cd78491d801f93e060b3ae0ac83.nc', '(MARÇO 2023 - 16a31) c769088bb86ee3cee8a4fae9a26db382.nc', '(NOVEMBRO 2024 - 1a15) 73449947fd760b6c7e864f371adc2ef.nc', '(DEZEMBRO 2024 - 1a15)

/tmp/ipython-input-2195183609.py:39: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'valid_time' ('valid_time',) The recommendation is to set join explicitly for this case.
  dados = xr.open_mfdataset(
/tmp/ipython-input-2195183609.py:39: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'latitude' ('latitude',) The recommendation is to set join explicitly for this case.
  dados = xr.open_mfdataset(



⚠️ Erro na combinação — tentando modo 'nested' (concat por tempo)...


/tmp/ipython-input-2195183609.py:48: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'latitude' ('latitude',) The recommendation is to set join explicitly for this case.
  dados = xr.open_mfdataset(
/tmp/ipython-input-2195183609.py:48: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'longitude' ('longitude',) The recommendation is to set join explicitly for this case.
  dados = xr.open_mfdataset(



✅ Dataset carregado com sucesso!



<xarray.Dataset> Size: 433MB
Dimensions:     (valid_time: 40920, latitude: 12, longitude: 22)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 327kB 2023-02-16 ... 2021-11-30T2...
  * latitude    (latitude) float64 96B -24.2 -24.2 -24.1 ... -23.4 -23.3 -23.2
  * longitude   (longitude) float64 176B -47.2 -47.19 -47.1 ... -46.2 -46.19
    number      int64 8B 0
    expver      (valid_time) <U4 655kB dask.array<chunksize=(312,), meta=np.ndarray>
Data variables:
    d2m         (valid_time, latitude, longitude) float32 43MB dask.array<chunksize=(312, 11, 11), meta=np.ndarray>
    t2m         (valid_time, latitude, longitude) float32 43MB dask.array<chunksize=(312, 11, 11), meta=np.ndarray>
    slhf        (valid_time, latitude, longitude) float32 43MB dask.array<chunksize=(312, 11, 11), meta=np.ndarray>
    sshf        (valid_time, latitude, longitude) float32 43MB dask.array<chunksize=(312, 11, 11), meta=np.ndarray>
    ssrd        (valid_time, latitude, longitude) float32 43MB dask.array<chunksize=(312, 11, 11), meta=np.ndarray>
    e           (valid_time, latitude, longitude) float32 43MB dask.array<chunksize=(312, 11, 11), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 43MB dask.array<chunksize=(312, 11, 11), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 43MB dask.array<chunksize=(312, 11, 11), meta=np.ndarray>
    sp          (valid_time, latitude, longitude) float32 43MB dask.array<chunksize=(312, 11, 11), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 43MB dask.array<chunksize=(312, 11, 11), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-10-27T15:01 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
df = dados.to_dataframe()
df = df.dropna()

In [4]:
df

d2m         t2m        slhf  \
valid_time          latitude longitude                                       
2023-02-16 00:00:00 -24.201  -47.1970   296.509644  298.026489 -16533335.0   
                             -47.0969   296.818604  297.999634 -14355036.0   
                             -46.9968   297.534546  298.710571 -14105576.0   
                             -46.8967   298.038574  299.397461 -13669884.0   
                    -24.100  -47.1970   296.106567  297.605957 -14921774.0   
...                                            ...         ...         ...   
2021-11-30 23:00:00 -23.200  -46.5964   291.750000  293.247864  -8395827.0   
                             -46.4963   291.350708  293.047668  -9538354.0   
                             -46.3962   291.256958  293.315369 -10076260.0   
                             -46.2961   291.351929  293.917419 -10015430.0   
                             -46.1960   291.995605  294.262207  -8253381.5   

                                              sshf        ssrd         e  \
valid_time          latitude longitude                                     
2023-02-16 00:00:00 -24.201  -47.1970   1038747.25  23305958.0 -0.006611   
                             -47.0969   -634686.50  22743906.0 -0.005740   
                             -46.9968    -61447.75  22206286.0 -0.005640   
                             -46.8967     81184.00  21603476.0 -0.005466   
                    -24.100  -47.1970   -659786.75  23067790.0 -0.005967   
...                                            ...         ...       ...   
2021-11-30 23:00:00 -23.200  -46.5964   -894719.75  13324076.0 -0.003357   
                             -46.4963    104558.25  13394760.0 -0.003814   
                             -46.3962    453649.75  13635064.0 -0.004029   
                             -46.2961    828792.50  13855776.0 -0.004005   
                             -46.1960   -314240.25  14076276.0 -0.003300   

                                             u10       v10           sp  \
valid_time          latitude longitude                                    
2023-02-16 00:00:00 -24.201  -47.1970  -0.910019 -0.254879   97994.7500   
                             -47.0969  -1.069443  0.105839   97992.0000   
                             -46.9968  -1.289597  0.464542   99240.0000   
                             -46.8967  -1.642563  0.627811  100490.7500   
                    -24.100  -47.1970  -0.121933 -0.558651   96682.7500   
...                                          ...       ...          ...   
2021-11-30 23:00:00 -23.200  -46.5964   0.404678 -1.563331   91031.0625   
                             -46.4963   0.480545 -1.582984   90487.3125   
                             -46.3962   0.421707 -1.463111   90720.5625   
                             -46.2961   0.298538 -1.348121   91458.3125   
                             -46.1960   0.127457 -1.182105   92382.0625   

                                              tp  number expver  
valid_time          latitude longitude                           
2023-02-16 00:00:00 -24.201  -47.1970   0.008061       0   0001  
                             -47.0969   0.008837       0   0001  
                             -46.9968   0.010821       0   0001  
                             -46.8967   0.010666       0   0001  
                    -24.100  -47.1970   0.004695       0   0001  
...                                          ...     ...    ...  
2021-11-30 23:00:00 -23.200  -46.5964   0.015270       0   0001  
                             -46.4963   0.011913       0   0001  
                             -46.3962   0.010566       0   0001  
                             -46.2961   0.008402       0   0001  
                             -46.1960   0.006649       0   0001  

[4378412 rows x 12 columns]

In [5]:

# --- 2️⃣ Verificar se há a coordenada temporal ---
if "valid_time" not in dados.coords:
    raise ValueError("❌ O dataset não contém uma coordenada temporal chamada 'valid_time'.")

# --- 3️⃣ Converter o eixo temporal para pandas.DatetimeIndex ---
tempo = pd.to_datetime(dados["valid_time"].values)

# --- 4️⃣ Criar DataFrame com componentes temporais ---
df_tempo = pd.DataFrame({
    "timestamp": tempo,
    "timestamp_segundos": tempo.view("int64"),   # segundos desde 1970
    "ano": tempo.year,
    "mes": tempo.month,
    "dia": tempo.day,
    "hora": tempo.hour,
    "minuto": tempo.minute,
    "segundo": tempo.second,
    "dia_semana": tempo.dayofweek,
    "dia_do_ano": tempo.dayofyear
})

#print(df_tempo.head())

# Exemplo: seleciona uma variável e um período
# Sort the dataset by valid_time before slicing
dados_sorted = dados.sortby('valid_time')
dados_filtrado = dados_sorted

# Converte para pandas sem estourar RAM
df_panda = dados_filtrado.to_dataframe().reset_index()
df_panda = df_panda.dropna()

# Junta com df_tempo
df_final = pd.merge(
    df_panda,
    df_tempo,
    left_on='valid_time',
    right_on='timestamp',
    how='left'
)

df_panda

,valid_time,latitude,longitude,d2m,t2m,slhf,sshf,ssrd,e,u10,v10,sp,tp,number,expver
23,2020-01-01 00:00:00,-24.2,-47.1900,295.882446,296.606445,-12626415.0,-5374780.5,27156416.0,-0.005049,-1.368388,-0.228513,97930.375,0.000336,0,0001
25,2020-01-01 00:00:00,-24.2,-47.0900,295.884033,296.721313,-13061472.0,-4630067.0,27113798.0,-0.005223,-1.634807,-0.463254,97935.625,0.000315,0,0001
27,2020-01-01 00:00:00,-24.2,-46.9900,296.079102,297.012085,-12878046.0,-4912239.0,27059548.0,-0.005150,-2.066936,-0.630246,99164.375,0.000344,0,0001
29,2020-01-01 00:00:00,-24.2,-46.8900,296.284790,297.303894,-12609391.0,-5259170.0,27010644.0,-0.005042,-2.459788,-0.772641,100463.125,0.000375,0,0001
45,2020-01-01 00:00:00,-24.1,-47.1900,295.862305,296.350220,-13214768.0,-4408237.5,27057096.0,-0.005284,-1.229838,-0.336697,96580.875,0.000525,0,0001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10802870,2024-12-31 23:00:00,-23.2,-46.5964,291.250549,293.717224,-10255067.0,-4246329.0,24612220.0,-0.004101,-2.006104,0.661022,91489.500,0.002697,0,0001
10802872,2024-12-31 23:00:00,-23.2,-46.4963,291.174194,293.420593,-9786872.0,-5151808.0,24490392.0,-0.003913,-1.859314,0.597393,90996.000,0.002424,0,0001
10802874,2024-12-31 23:00:00,-23.2,-46.3962,291.230957,293.562927,-9981260.0,-4897497.5,24498628.0,-0.003991,-1.728638,0.578381,91276.000,0.002353,0,0001
10802876,2024-12-31 23:00:00,-23.2,-46.2961,291.578003,293.923645,-10155309.0,-4563473.5,24513856.0,-0.004061,-1.633911,0.573956,92056.250,0.002304,0,0001


In [6]:
print(np.unique(df_panda['latitude'].values))
print(np.unique(df_panda['longitude'].values))
print(len(np.unique(df_panda['latitude'].values)))
print(len(np.unique(df_panda['longitude'].values)))

[-24.201 -24.2   -24.1   -24.    -23.9   -23.8   -23.7   -23.6   -23.5
 -23.4   -23.3   -23.2  ]
[-47.197  -47.19   -47.0969 -47.09   -46.9968 -46.99   -46.8967 -46.89
 -46.7966 -46.79   -46.6965 -46.69   -46.5964 -46.59   -46.4963 -46.49
 -46.3962 -46.39   -46.2961 -46.29   -46.196  -46.19  ]
12
22


In [7]:
tempo = np.unique(df_panda['valid_time'].values)
id = np.where(df_panda['valid_time'].values == tempo[13])
print(len(df_panda['d2m'].values[id]))
print(len(np.unique(df_panda['latitude'].values)) * len(np.unique(df_panda['longitude'].values)))

107
264


In [8]:
from tqdm import tqdm

df_panda.keys()

Index(['valid_time', 'latitude', 'longitude', 'd2m', 't2m', 'slhf', 'sshf',
       'ssrd', 'e', 'u10', 'v10', 'sp', 'tp', 'number', 'expver'],
      dtype='object')

In [9]:
import numpy as np
import pandas as pd

def calcula_vapor_umidade(data):
    # Temperatura e ponto de orvalho em Celsius
    t2m_celsius = data["t2m"] - 273.15
    d2m_celsius = data["d2m"] - 273.15

    # 1) Pressão de vapor de saturação (es) em Pa
    es = 610.94 * np.exp(17.625 * t2m_celsius / (243.04 + t2m_celsius))

    # 2) Pressão de vapor real (ev) em Pa
    ev = 610.94 * np.exp(17.625 * d2m_celsius / (243.04 + d2m_celsius))

    # 3) Umidade Relativa (%)
    RH = 100 * ev / es

    # 4) Déficit de Pressão de Vapor (kPa)
    VPD = (es - ev) / 1000

    # Retorna como DataFrame para fácil concatenação
    return pd.DataFrame({
        "es": es,
        "ev": ev,
        "RH": RH,
        "VPD": VPD
    })

# --- Aplicar ao DataFrame original ---
resultados = calcula_vapor_umidade(df_panda)

# Adiciona novas colunas ao df_panda
df_panda = pd.concat([df_panda.reset_index(drop=True), resultados.reset_index(drop=True)], axis=1)


In [10]:
df_panda

,valid_time,latitude,longitude,d2m,t2m,slhf,sshf,ssrd,e,u10,v10,sp,tp,number,expver,es,ev,RH,VPD
0,2020-01-01 00:00:00,-24.2,-47.1900,295.882446,296.606445,-12626415.0,-5374780.5,27156416.0,-0.005049,-1.368388,-0.228513,97930.375,0.000336,0,0001,2882.212300,2758.732488,95.715797,0.123480
1,2020-01-01 00:00:00,-24.2,-47.0900,295.884033,296.721313,-13061472.0,-4630067.0,27113798.0,-0.005223,-1.634807,-0.463254,97935.625,0.000315,0,0001,2902.241666,2758.997990,95.064378,0.143244
2,2020-01-01 00:00:00,-24.2,-46.9900,296.079102,297.012085,-12878046.0,-4912239.0,27059548.0,-0.005150,-2.066936,-0.630246,99164.375,0.000344,0,0001,2953.488629,2791.805311,94.525683,0.161683
3,2020-01-01 00:00:00,-24.2,-46.8900,296.284790,297.303894,-12609391.0,-5259170.0,27010644.0,-0.005042,-2.459788,-0.772641,100463.125,0.000375,0,0001,3005.713114,2826.768353,94.046512,0.178945
4,2020-01-01 00:00:00,-24.1,-47.1900,295.862305,296.350220,-13214768.0,-4408237.5,27057096.0,-0.005284,-1.229838,-0.336697,96580.875,0.000525,0,0001,2837.970221,2755.364595,97.089271,0.082606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378407,2024-12-31 23:00:00,-23.2,-46.5964,291.250549,293.717224,-10255067.0,-4246329.0,24612220.0,-0.004101,-2.006104,0.661022,91489.500,0.002697,0,0001,2416.657345,2072.784398,85.770720,0.343873
4378408,2024-12-31 23:00:00,-23.2,-46.4963,291.174194,293.420593,-9786872.0,-5151808.0,24490392.0,-0.003913,-1.859314,0.597393,90996.000,0.002424,0,0001,2372.820067,2062.863856,86.937222,0.309956
4378409,2024-12-31 23:00:00,-23.2,-46.3962,291.230957,293.562927,-9981260.0,-4897497.5,24498628.0,-0.003991,-1.728638,0.578381,91276.000,0.002353,0,0001,2393.766924,2070.234852,86.484395,0.323532
4378410,2024-12-31 23:00:00,-23.2,-46.2961,291.578003,293.923645,-10155309.0,-4563473.5,24513856.0,-0.004061,-1.633911,0.573956,92056.250,0.002304,0,0001,2447.580582,2115.805985,86.444794,0.331775


In [11]:
#RODE APENAS SE NÃO TIVER OS DADOS PRONTOS
from tqdm import tqdm
import numpy as np
import pandas as pd

def criar_dados(df_panda):
  data = []

  # Lista de variáveis a processar
  variaveis = ['d2m', 't2m', 'u10', 'v10', 'slhf', 'sshf', 'ssrd', 'sp', 'e', 'tp', 'es', 'ev', 'RH', 'VPD']

  for t in tqdm(tempo, desc="Processando tempos"):
      # Filtra as linhas com o tempo atual
      mask = df_panda['valid_time'].values == t

      # Dicionário para armazenar os resultados deste tempo
      stats = {'tempo': t}

      for var in variaveis:
          valores = df_panda[var].values[mask]

          # Se for precipitação ou evaporação, somar; caso contrário, calcular média/mín/máx
          if var in ['e', 'tp']:
              stats[f'{var}_sum'] = np.sum(valores)
          else:
              stats[f'{var}_mean'] = np.mean(valores)
              stats[f'{var}_min'] = np.min(valores)
              stats[f'{var}_max'] = np.max(valores)

      data.append(stats)

  # Converte para DataFrame
  df_resumo = pd.DataFrame(data)
  return df_resumo

In [12]:
#RODE APENAS SE NÃO TIVER OS DADOS PRONTOS
#df_resumo = criar_dados(df_panda)
#df_resumo.to_csv('teste.csv', index=False, encoding='utf-8')

In [15]:
df_resumo = pd.read_csv('teste.csv')

In [16]:
df_resumo

,tempo,d2m_mean,d2m_min,d2m_max,t2m_mean,t2m_min,t2m_max,u10_mean,u10_min,u10_max,...,ev_mean,ev_min,ev_max,RH_mean,RH_min,RH_max,VPD_mean,VPD_min,VPD_max,data
0,2020-01-01 00:00:00,294.35715,290.97913,296.40527,296.00278,294.86487,298.34375,-1.671670,-2.459788,-0.666270,...,2520.350811,2037.708369,2847.425769,90.793529,64.118763,98.381075,0.260865,0.043199,1.147642,2020-01-01
1,2020-01-01 01:00:00,294.31824,292.10450,296.99230,295.47670,294.04517,297.61250,-1.653717,-2.406607,-0.626455,...,2513.622883,2186.621767,2949.978385,93.364740,72.843492,99.174344,0.181176,0.020972,0.825427,2020-01-01
2,2020-01-01 02:00:00,294.07920,292.05470,296.87305,295.05322,293.46850,297.44598,-1.352155,-2.164139,-0.337479,...,2476.886533,2179.834825,2928.885907,94.335643,78.891616,99.628635,0.149498,0.009513,0.613241,2020-01-01
3,2020-01-01 03:00:00,293.72958,291.85193,296.77880,294.66718,292.97913,297.28674,-0.911181,-1.818641,0.163568,...,2424.424361,2152.395124,2912.312676,94.491535,82.287300,99.735729,0.140987,0.006596,0.484505,2020-01-01
4,2020-01-01 04:00:00,293.31630,291.23560,296.54980,294.25015,292.60730,297.04760,-0.436056,-1.570677,0.646486,...,2364.066126,2070.838234,2872.380601,94.464307,86.122503,99.761469,0.136942,0.006171,0.368936,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16232,2021-11-07 08:00:00,287.66873,285.96313,290.62463,288.49414,287.35474,291.38184,-1.134926,-2.489264,-0.402289,...,1654.251242,1476.839231,1992.682936,94.889116,83.226693,99.172560,0.088843,0.014362,0.304231,2021-11-07
16233,2021-11-07 09:00:00,287.72272,285.99585,290.81494,288.70090,287.45886,291.59766,-1.111973,-2.205138,-0.371154,...,1660.556588,1480.003787,2016.744777,93.964497,83.077170,98.707887,0.106051,0.023375,0.309700,2021-11-07
16234,2021-11-07 10:00:00,287.45624,285.97852,290.61304,290.16675,288.84485,293.13794,-1.599028,-2.932492,-0.733213,...,1632.121921,1478.326302,1991.224878,84.108701,75.877674,87.584847,0.307637,0.226625,0.486916,2021-11-07
16235,2021-11-07 11:00:00,287.45157,286.11792,290.37305,291.62152,290.39330,293.99200,-1.727462,-2.960196,-0.887259,...,1630.652534,1491.864602,1961.260374,76.733765,68.628197,80.776550,0.493691,0.385587,0.715768,2021-11-07


In [17]:
df_resumo['tempo'] = pd.to_datetime(df_resumo['tempo'])
df_resumo['data'] = df_resumo['tempo'].dt.date
df_diario = (
    df_resumo
    .groupby('data')
    .agg({
        'd2m_mean': 'mean',
        'd2m_min': 'min',
        'd2m_max': 'max',
        't2m_mean': 'mean',
        't2m_min': 'min',
        't2m_max': 'max',
        'u10_mean': 'mean',
        'u10_min': 'min',
        'u10_max': 'max',
        'v10_mean': 'mean',
        'v10_min': 'min',
        'v10_max': 'max',
        'slhf_mean': 'mean',
        'sshf_mean': 'mean',
        'ssrd_mean': 'mean',
        'sp_mean': 'mean',
        'e_sum': 'sum',
        'tp_sum': 'sum',
        'es_mean': 'mean',
        'ev_mean': 'mean',
        'RH_mean': 'mean',
        'VPD_mean': 'mean',
        'es_min': 'min',
        'es_max': 'max',
        'ev_min': 'min',
        'ev_max': 'max',
        'RH_min': 'min',
        'RH_max': 'max',
        'VPD_min': 'min',
        'VPD_max': 'max'
    })
    .reset_index()
)
df_diario

,data,d2m_mean,d2m_min,d2m_max,t2m_mean,t2m_min,t2m_max,u10_mean,u10_min,u10_max,...,RH_mean,VPD_mean,es_min,es_max,ev_min,ev_max,RH_min,RH_max,VPD_min,VPD_max
0,2020-01-01,292.666795,287.50293,300.11768,298.356312,291.25793,305.37952,0.457941,-3.303340,3.354943,...,73.248825,0.988804,2073.746157,4810.687691,1632.416453,3552.257718,36.539641,99.761469,0.006171,2.871634
1,2020-01-02,293.406096,289.85010,299.29200,296.250915,292.52783,301.23670,0.799427,-1.667494,3.300402,...,84.489326,0.456947,2245.063660,3792.636542,1897.334838,3383.532998,61.348478,99.467196,0.012338,1.377248
2,2020-01-03,292.296292,290.12110,296.03943,294.345430,291.69934,299.01294,0.123197,-1.924273,3.471870,...,88.564353,0.303775,2131.945548,3328.116254,1930.229150,2785.105443,66.349927,99.955850,0.001040,1.110049
3,2020-01-04,291.970090,289.90393,296.24402,293.869149,290.25757,298.85193,-0.046169,-1.935276,2.982586,...,89.195695,0.279918,1946.983749,3296.503784,1903.829757,2819.807683,66.990279,99.653280,0.007672,1.088167
4,2020-01-05,292.203271,289.97754,295.42230,294.359055,290.07240,300.24730,-0.298716,-2.821115,2.675707,...,88.213359,0.336362,1924.280393,3579.405823,1912.742182,2682.684612,68.176722,99.899298,0.002168,1.133866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,2021-11-03,289.351875,286.66296,293.06677,292.529837,288.29760,299.91736,-1.920357,-4.432738,-0.113837,...,83.520865,0.433720,1718.212508,3510.662357,1545.850368,2321.444366,46.597629,99.432211,0.010604,1.857099
673,2021-11-04,289.432407,285.96692,293.46777,293.012911,286.26868,301.17688,-1.183235,-3.348328,1.216293,...,81.721659,0.517760,1506.628157,3779.437309,1477.204975,2379.745546,44.231037,99.453342,0.009552,2.107753
674,2021-11-05,289.911327,287.24304,293.03455,293.275084,287.30652,300.58948,-1.224774,-3.864059,0.434189,...,82.612511,0.482936,1611.801353,3651.932075,1605.188081,2316.813842,50.773893,99.984276,0.000265,1.797704
675,2021-11-06,288.512395,284.87036,291.88360,292.520979,286.39307,301.41430,-1.182546,-3.073519,0.605736,...,80.032756,0.548377,1518.906276,3832.067751,1374.498962,2156.661973,40.996403,99.899959,0.001654,2.257807


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score, PredictionErrorDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [19]:
df_diario['tp_sum'] = df_diario['tp_sum'].shift(-1) #determina previsão de chuva no dia seguinte IMPORTANTE: rodar apenas uma unica vez
df_diario = df_diario.dropna()

In [20]:
#X = df_diario.drop(['data', 'tp_sum'], axis=1)
#y = df_diario['tp_sum']

ind_sem_chuva = np.where(df_diario['tp_sum']<=0.1)[0]
ind_com_chuva = np.where(df_diario['tp_sum']>0.1)[0]

scaler = StandardScaler()
# Drop the 'data' column as it's not numerical
X_scaled = scaler.fit_transform(df_diario.drop('data', axis=1))
nomes = df_diario.drop('data', axis=1).columns
df_scaled = pd.DataFrame(X_scaled, columns=nomes)

df_diario_sem_chuva = df_scaled.iloc[ind_sem_chuva]
df_diario_com_chuva = df_scaled.iloc[ind_com_chuva]

df_diario_sem_chuva['tp_sum'] = 0
df_diario_com_chuva['tp_sum'] = 1

X_sem_chuva = df_diario_sem_chuva.drop(['tp_sum'], axis=1)
y_sem_chuva = df_diario_sem_chuva['tp_sum']

X_com_chuva = df_diario_com_chuva.drop(['tp_sum'], axis=1)
y_com_chuva = df_diario_com_chuva['tp_sum']

X_train_sem_chuva, X_test_sem_chuva, y_train_sem_chuva, y_test_sem_chuva = train_test_split(X_sem_chuva, y_sem_chuva, test_size=0.2, random_state=42)
X_train_com_chuva, X_test_com_chuva, y_train_com_chuva, y_test_com_chuva = train_test_split(X_com_chuva, y_com_chuva, test_size=0.2, random_state=42)

X_train = pd.concat([X_train_sem_chuva, X_train_com_chuva])
X_test = pd.concat([X_test_sem_chuva, X_test_com_chuva])
y_train = pd.concat([y_train_sem_chuva, y_train_com_chuva])
y_test = pd.concat([y_test_sem_chuva, y_test_com_chuva])

#

/tmp/ipython-input-2964317081.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diario_sem_chuva['tp_sum'] = 0
/tmp/ipython-input-2964317081.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diario_com_chuva['tp_sum'] = 1


In [21]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [22]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nMatriz de confusão:\n", confusion_matrix(y_test, y_pred))
print("\nRelatório de classificação:\n", classification_report(y_test, y_pred))

Acurácia: 0.9044117647058824

Matriz de confusão:
 [[ 21  11]
 [  2 102]]

Relatório de classificação:
               precision    recall  f1-score   support

           0       0.91      0.66      0.76        32
           1       0.90      0.98      0.94       104

    accuracy                           0.90       136
   macro avg       0.91      0.82      0.85       136
weighted avg       0.91      0.90      0.90       136

